In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [5]:
DRIVER = None
def initialize_driver():
    global DRIVER
    if DRIVER is None:
        print('Initiating driver...')
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('-headless') 
        chrome_options.add_argument('-no-sandbox') 
        DRIVER = webdriver.Chrome('chromedriver',options=chrome_options)  # Create the new chrome browser with specific options
        print('Finished!')

def close_driver():
    global DRIVER
    if not DRIVER is None:
        print("Quitting driver...")
        DRIVER.quit()
        print("Done!")

    DRIVER = None

initialize_driver()

DRIVER

Initiating driver...
Finished!


<selenium.webdriver.chrome.webdriver.WebDriver (session="c9a29c014a5e7e5ebd0be4c493fef694")>

In [6]:
DRIVER.get('https://vnexpress.net/')
DRIVER.current_url

'https://vnexpress.net/'

In [13]:
all_news_elements = DRIVER.find_elements(By.CLASS_NAME, 'item-news')
len(all_news_elements)
news_element = all_news_elements[10]
print(news_element.get_attribute('outerHTML'))

<article class="item-news item-news-common " data-id="4504759">
<h3 class="title-news">
<a href="https://vnexpress.net/dong-dat-o-quang-ngai-4504759.html" title="Động đất ở Quảng Ngãi" data-medium="Item-11" data-thumb="1" data-itm-source="#vn_source=Home&amp;vn_campaign=ThuongVien&amp;vn_medium=Item-11&amp;vn_term=Desktop&amp;vn_thumb=1" data-itm-added="1">Động đất ở Quảng Ngãi</a>
</h3>
<div class="thumb-art">
<a href="https://vnexpress.net/dong-dat-o-quang-ngai-4504759.html" class="thumb thumb-5x3" title="Động đất ở Quảng Ngãi" data-medium="Item-11" data-thumb="1" data-itm-source="#vn_source=Home&amp;vn_campaign=ThuongVien&amp;vn_medium=Item-11&amp;vn_term=Desktop&amp;vn_thumb=1" data-itm-added="1">
<picture>
<!--[if IE 9]><video style="display: none;"><![endif]-->
<source data-srcset="https://i1-vnexpress.vnecdn.net/2022/08/28/dongdat-1661657439-6814-1661657445.jpg?w=220&amp;h=132&amp;q=100&amp;dpr=1&amp;fit=crop&amp;s=b9z2HRH1OhD62Q9bXlFe_Q 1x, https://i1-vnexpress.vnecdn.net/2022/

In [18]:
description_element = news_element.find_element(By.CLASS_NAME, 'description')
a_element = description_element.find_element(By.TAG_NAME, 'a')
article_link = a_element.get_attribute('href')
print(article_link)
title = a_element.get_attribute('title')
print(title)
a_element.text

https://vnexpress.net/dong-dat-o-quang-ngai-4504759.html
Động đất ở Quảng Ngãi


'Trận động đất mạnh 2,5 độ xảy ra ở xã Sơn Kỳ, huyện Sơn Hà, độ sâu chấn tiêu 10 km, sáng 28/8.'

In [24]:
def get_link_title_description(news_element):
    '''
    Return link, title and description of an article from a web element
    '''
    
    description_element = news_element.find_element(By.CLASS_NAME, 'description')
    a_element           = description_element.find_element(By.TAG_NAME, 'a')

    # article link
    article_link = a_element.get_attribute('href')

    # title
    title = a_element.get_attribute('title')

    # description
    desc = a_element.text

    return article_link, title, desc

news_element = all_news_elements[10]

a, b, c = get_link_title_description(news_element)
a, b, c

('https://vnexpress.net/dong-dat-o-quang-ngai-4504759.html',
 'Động đất ở Quảng Ngãi',
 'Trận động đất mạnh 2,5 độ xảy ra ở xã Sơn Kỳ, huyện Sơn Hà, độ sâu chấn tiêu 10 km, sáng 28/8.')

In [25]:
thumbnail_link = news_element.find_element(By.TAG_NAME, 'img').get_attribute('src')
print(thumbnail_link)

https://i1-vnexpress.vnecdn.net/2022/08/28/dongdat-1661657439-6814-1661657445.jpg?w=220&h=132&q=100&dpr=1&fit=crop&s=b9z2HRH1OhD62Q9bXlFe_Q


In [30]:
def get_thumbnail_link(news_element):
    '''
    Return thumbnail link (if possible) given the web element
    '''
    
    thumbnail_link = ''
    try:
        thumbnail_link = news_element.find_element(By.TAG_NAME, 'img').get_attribute('src')
    except Exception:  # if there's an error
        print('Cannot find thumbnail_link')
    
    return thumbnail_link

news_element = all_news_elements[10]
print(get_thumbnail_link(news_element))

https://i1-vnexpress.vnecdn.net/2022/08/28/dongdat-1661657439-6814-1661657445.jpg?w=220&h=132&q=100&dpr=1&fit=crop&s=b9z2HRH1OhD62Q9bXlFe_Q


In [31]:
close_driver()
initialize_driver()

Quitting driver...
Done!
Initiating driver...
Finished!


In [32]:
DRIVER.get('https://vnexpress.net/')

In [38]:
def scrape_vnexpress(DRIVER): 
    all_news_elements = DRIVER.find_elements(By.CLASS_NAME, 'item-news')
    results = []
    n_skip = 0
    n_success = 0
    for news_element in all_news_elements:
        def get_link_title_description(news_element):
            description_element = news_element.find_element(By.CLASS_NAME, 'description')
            a_element           = description_element.find_element(By.TAG_NAME, 'a')
            article_link = a_element.get_attribute('href')
            title = a_element.get_attribute('title')
            desc = a_element.text
            return article_link, title, desc
        try:
            article_link, title, desc = get_link_title_description(news_element) # must have 3
            n_success += 1
        except Exception:
            print('Cannot get this article. Skip')
            n_skip += 1
            continue
        
        def get_thumbnail_link(news_element):
            thumbnail_link = ''
            try:
                thumbnail_link = news_element.find_element(By.TAG_NAME, 'img').get_attribute('src')
            except Exception:  # if there's an error
                print('Cannot find thumbnail_link')

        results.append([article_link, title, desc, thumbnail_link])

    print(f'Number of scraped articles: {n_success}')
    print(f'Number of skipped articles: {n_skip}')       
    return results

results = scrape_vnexpress(DRIVER)

Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get this article. Skip
Cannot get

In [39]:
results[:5]

[['https://vnexpress.net/ca-covid-tai-tp-hcm-tang-nguy-co-xuat-hien-lan-song-dich-moi-4504772.html',
  'Ca Covid tại TP HCM tăng, nguy cơ xuất hiện làn sóng dịch mới',
  'TP HCM ghi nhận hơn 1.100 ca Covid mới trong tuần qua, tăng 258 ca so với tuần trước, số bệnh nhân nặng tại các bệnh viện cũng ở mức cao.',
  'https://i1-vnexpress.vnecdn.net/2022/08/28/dongdat-1661657439-6814-1661657445.jpg?w=220&h=132&q=100&dpr=1&fit=crop&s=b9z2HRH1OhD62Q9bXlFe_Q'],
 ['https://vnexpress.net/shinkansen-o-viet-nam-4504662.html',
  'Shinkansen ở Việt Nam?',
  'Đường sắt cao tốc theo mô hình Shinkansen có ít nhất bốn khiếm khuyết, không phù hợp áp dụng cho tuyến Hà Nội - TP HCM.',
  'https://i1-vnexpress.vnecdn.net/2022/08/28/dongdat-1661657439-6814-1661657445.jpg?w=220&h=132&q=100&dpr=1&fit=crop&s=b9z2HRH1OhD62Q9bXlFe_Q'],
 ['https://vnexpress.net/ong-jonathan-hanh-nguyen-ipp-air-cargo-chua-duoc-cap-phep-da-co-don-hang-4503417.html',
  "Ông Jonathan Hạnh Nguyễn: 'IPP Air Cargo chưa được cấp phép đã có 

In [40]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'